In [28]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [29]:
from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [30]:
def calculate_critic(a, b):
    
    """
        This function calculates the critic point 
        (p_c, v_c, T_c) from given a and b parameters of 
        the Van der Waals equation of state for real gases.
        
        :math:`(P + a \\frac{n^2}{V^2})(V - nb) = nRT`
        
        :math:`p_c = \\frac{a}{27 b^2}`
        :math:`v_c = 3b`
        :math:`T_c = \\frac{8a}{27 b R}`
        
   Args:
       a: Term related with the attraction between particles in
       L^2 bar/mol^2.\n
       b: Term related with the volume that is occupied by one 
       mole of the molecules in L/mol.\n
       
   Returns:
       p_c: Critical pressure in bar.\n
       v_c: Critical volume in L/mol.\n
       T_c: Critical tenperature in K.\n
        
    """
    
    if b == 0.0:
        return None
    
    k_B = 1.3806488e-23 #m^2 kg s^-2 K^-1
    N_A = 6.02214129e23 
    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1
    
    p_c = a/27.0/(b**2)
    v_c = 3.0*b
    T_c = 8.0*a/27.0/b/R
    
    return p_c, v_c, T_c

In [4]:
def get_absolute_isotherms(a, b, v_values, T_values):
    #given v (1d array) and T (Nd array) calculate N isotherm arrays 
    
    isotherms = []
    
    #N_A = 6.02214129e23 

    R = 0.082 * 1.01325 #atm L mol^-1 K^-1 #now in bar L mol^-1 K^-1

    for T in T_values:
        isot = []
        for v in v_values:
            p = R*T/(v - b) - (a/v**2)
            isot.append(p)
            
        isotherms.append(isot)
        
    return isotherms

In [7]:
def get_zoom_arrays(initial_x_range, final_x_range, initial_y_range, final_y_range, size):
    
    x_min = np.linspace(min(initial_x_range), min(final_x_range), size)
    x_max = np.linspace(max(initial_x_range), max(final_x_range), size)
    
    y_min = np.linspace(min(initial_y_range), min(final_y_range), size)
    y_max = np.linspace(max(initial_y_range), max(final_y_range), size)
    
    return x_min, x_max, y_min, y_max

In [20]:
def update_scales(change):
    
    index = change.owner.value
    scale_x_118_001.min, scale_x_118_001.max, scale_y_118_001.min, scale_y_118_001.max = x_min[index], x_max[index], y_min[index], y_max[index]

In [36]:
def bar_to_atm(p_values):
    
    for i in range(len(p_values)):
        p_values[i] = np.array(p_values[i]) * 0.9869 #unit change from bar to atm
        
    return p_values

In [43]:
#In this program we are going to use water's parameters
a = 5.536 #L^2 bar mol^-2
b = 0.03049 #L mol^-1

colors = ['#0079c4','#f09205','#21c400', '#850082']

p_c, v_c, T_c = calculate_critic(a, b)

p_c = p_c * 0.9869 #unit change from bar to atm

v_values = np.linspace(-20, 20, 3000) #L/mol
T_values = [0.9*T_c, 1.0*T_c, 1.1*T_c]

p_values = get_absolute_isotherms(a, b, v_values, T_values)
p_values = bar_to_atm(p_values)

v_values_hd = np.linspace(-20, 20, 10000)
p_values_hd = get_absolute_isotherms(a, b, v_values_hd, T_values)

#####################
######TOP BLOCK######
#####################

top_block = widgets.VBox([], layout=widgets.Layout(align_items='center'))

scale_x_118_001 = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y_118_001 = bqs.LinearScale(min = min(p_values[1]), max = max(p_values[1]))

axis_x_118_001 = bqa.Axis(
                    scale=scale_x_118_001,
                    tick_format='.1f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_118_001 = bqa.Axis(
                scale=scale_y_118_001,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_118_001 = Figure(
                title='Van der Waals equation of state',
                marks=[],
                axes=[axis_x_118_001, axis_y_118_001],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout()
                      )

tb_118_001 = Toolbar(figure=fig_118_001, layout=widgets.Layout(align_self='center'))

marks = [bqm.Lines(
               x = [v_values for elem in p_values], 
               y = p_values, 
               scales = {'x': scale_x_118_001, 'y': scale_y_118_001}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_118_001.marks = marks

zoom_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=10000,
    step=1,
    description='Zoom:',
    disabled=False,
    continuous_update=True,
    orientation='vertical',
    readout=False,
    readout_format='d',
    layout = widgets.Layout(margin='35px 0 0 0', height='80%')
)

x_min, x_max, y_min, y_max = get_zoom_arrays([scale_x_118_001.min, scale_x_118_001.max],
                                            [0.0, 0.2],
                                            [scale_y_118_001.min, scale_y_118_001.max],
                                            [0.0, 456],
                                            zoom_slider.max+1)
zoom_slider.observe(update_scales, 'value')

top_block.children = [widgets.HBox([fig_118_001, zoom_slider]), tb_118_001]


#####################
####BOTTOM BLOCK#####
#####################

bottom_block = widgets.HBox([], layout=widgets.Layout(height='300px'))

scale_x_118_003 = bqs.LinearScale(min = min(v_values), max = max(v_values))
scale_y_118_003 = bqs.LinearScale(min = min(p_values[1]), max = max(p_values[1]))

axis_x_118_003 = bqa.Axis(
                    scale=scale_x_118_003,
                    tick_format='.0f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_118_003 = bqa.Axis(
                scale=scale_y_118_003,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

marks = [bqm.Lines(
               x = [v_values for elem in p_values], 
               y = p_values, 
               scales = {'x': scale_x_118_003, 'y': scale_y_118_003}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_118_003 = Figure(
                title='',
                marks=marks,
                axes=[axis_x_118_003, axis_y_118_003],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = False,
                layout = widgets.Layout(height='90%', width='25%')
                )



scale_x_118_004 = bqs.LinearScale(min = -2.0, max = 2.0)
scale_y_118_004 = bqs.LinearScale(min = -10*p_c, max = 10*p_c)

axis_x_118_004 = bqa.Axis(
                    scale=scale_x_118_004,
                    tick_format='.0f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_118_004 = bqa.Axis(
                scale=scale_y_118_004,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

marks = [bqm.Lines(
               x = [v_values_hd for elem in p_values_hd], 
               y = p_values_hd, 
               scales = {'x': scale_x_118_004, 'y': scale_y_118_004}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_118_004 = Figure(
                title='',
                marks=marks,
                axes=[axis_x_118_004, axis_y_118_004],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(height='90%', width='25%')
                )



scale_x_118_005 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_118_005 = bqs.LinearScale(min = 0.0, max = 10*p_c)

axis_x_118_005 = bqa.Axis(
                    scale=scale_x_118_005,
                    tick_format='.1f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_118_005 = bqa.Axis(
                scale=scale_y_118_005,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

marks = [bqm.Lines(
               x = [v_values_hd for elem in p_values_hd], 
               y = p_values_hd, 
               scales = {'x': scale_x_118_005, 'y': scale_y_118_005}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_118_005 = Figure(
                title='',
                marks=marks,
                axes=[axis_x_118_005, axis_y_118_005],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(height='90%', width='25%')
                )

scale_x_118_006 = bqs.LinearScale(min = 0.5*v_c, max = 2.0*v_c)
scale_y_118_006 = bqs.LinearScale(min = 0.0, max = 2.0*p_c)

axis_x_118_006 = bqa.Axis(
                    scale=scale_x_118_006,
                    tick_format='.1f',#'0.2f',
                    tick_style={'font-size': '15px'},
                    num_ticks=5,
                    grid_lines = 'none',
                    grid_color = '#8e8e8e', 
                    label='v (L/mol)',
                    label_location='middle',
                    label_style={'stroke': 'black', 'default-size': 35},
                    label_offset='50px')

axis_y_118_006 = bqa.Axis(
                scale=scale_y_118_006,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='p (atm)',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

marks = [bqm.Lines(
               x = [v_values_hd for elem in p_values_hd], 
               y = p_values_hd, 
               scales = {'x': scale_x_118_006, 'y': scale_y_118_006}, 
               opacities = [1.0],
               visible = True, #True, #t == '1.00',
               colors = colors,
               #labels = '',
               )]

fig_118_006 = Figure(
                title='',
                marks=marks,
                axes=[axis_x_118_006, axis_y_118_006],
                animation_duration=0, #500,
                #layout = widgets.Layout(align_self='center', width='75%'),
                legend_location='top-right',
                background_style= {'fill': 'white',  'stroke': 'black'},
                min_aspect_ratio=1.0,
                fig_margin=dict(top=50, bottom=60, left=80, right=30),
                toolbar = True,
                layout = widgets.Layout(height='90%', width='25%')
                )

bottom_block.children = [fig_118_003, fig_118_004, fig_118_005, fig_118_006]

#####################
####MAIN BLOCK#####
#####################

main_block = widgets.VBox([], layout=widgets.Layout(align_items='center'))

main_block.children = [top_block,
                      bottom_block]

main_block